In [ ]:
import photutils as pht
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob,os
from photutils import CircularAperture
from photutils.utils import calc_total_error
from astropy.table import Table
from astropy import table
from astropy.stats import SigmaClip
import pandas as pd


path =r"C:/Users/Alexander/Downloads/Practicas/CALIBRADAS/2022-05-13/"
#path =r"./"

final = pd.DataFrame(columns={'image','x','y','flux','flux_e'}, index=None)
for j in range(1,405):
    ma = '0'+str(j)
    
    if j <100 :
        ma = '00'+str(j)
    if j <10 :
        ma = '000'+str(j)
        
    print(ma)
    
    data=fits.getdata(path+'C_DB_TRES1b-'+ma+'_gSDSS.fit')
    
    ## here we can show some statistics about the sky
    mean, median, std = sigma_clipped_stats(data)

    ## or first mask sources then estimate the sky background
    mask = pht.make_source_mask(data, 3, 5)
    mean, median, std = sigma_clipped_stats(data, sigma=3.0, mask=mask)
    
    ## get 2D sky map
    sigma_clip = SigmaClip(sigma=3.)
    bkg_estimator = pht.SExtractorBackground()
    bkg = pht.Background2D(data, (64, 64), mask=mask,filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
     ## find objects and calculate basic information
    daofind = pht.IRAFStarFinder(fwhm=4.5, threshold=5.*bkg.background_rms_median,exclude_border=True, sharplo=
    0.5, sharphi=2.0, roundlo=0.0, roundhi=0.7)
    sources = daofind(data - bkg.background_median)
    
    x_min = 920
    x_max = 970
    y_min = 1000
    y_max = 1050
    if j <250:
        x_min = 960
        x_max = 1050
        
    for t in sources:
        if t['xcentroid'] > x_min and t['xcentroid'] <= x_max and t['ycentroid'] > y_min and t['ycentroid'] <= y_max:
            sources = t
    print(sources)
    if(len(sources)/12 >1):
        print(' MAYOR QUE 1 ' + str(len(sources)))
    else:
        positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
        apertures = CircularAperture(positions, r=8)
        plt.figure(figsize=(8,8))
        plt.imshow(data, cmap='Greys_r', origin='lower', vmin=300,vmax=600, interpolation='nearest')
        apertures.plot(color='red', lw=1.5, alpha=0.5)

        error=calc_total_error(data-bkg.background, bkg.background_rms, 1.85)

        radii=[8]
        positions=[sources['xcentroid'],sources['ycentroid']]
        apertures = [pht.CircularAperture(positions, r=r) for r in radii]
        aper_phot= pht.aperture_photometry(data - bkg.background, apertures, error=error)
        print(aper_phot)
        final = final.append({'image':ma,'x':aper_phot[0]['xcenter'],'y':aper_phot[0]['ycenter'],'flux':aper_phot[0]['aperture_sum'], 'flux_e':aper_phot[0]['aperture_sum_err']}, ignore_index=True)

final.to_csv('resultados.csv')
print(final)

0001
 id     xcentroid          ycentroid            fwhm            sharpness           roundness              pa        npix   sky          peak             flux                mag        
--- ------------------ ----------------- ------------------ ------------------ ------------------- ----------------- ---- -------- --------------- ------------------ -------------------
241 1019.9855751479574 1028.921508730478 2.9970522583628454 0.6660116129695212 0.03647285434765514 35.12127363199439   25 839.0137 8994.3544921875 132892.29272460938 -12.808749485384777
 id    xcenter       ycenter       aperture_sum    aperture_sum_err 
         pix           pix                                          
--- ------------- ------------- ----------------- ------------------
  1 1019.98557515 1028.92150873 284124.5447618532 482.22156104256214
0002
 id     xcentroid          ycentroid             fwhm          sharpness          roundness               pa        npix    sky          peak             fl

 id     xcentroid         ycentroid            fwhm           sharpness          roundness              pa         npix   sky          peak             flux               mag        
--- ------------------ ---------------- ----------------- ------------------ ------------------ ------------------ ---- -------- ---------------- ---------------- -------------------
200 1016.8650059318085 1028.96463442851 3.001955487562749 0.6671012194583886 0.0783368859618006 40.220421555403725   25 1825.437 5458.07177734375 83505.2099609375 -12.304283930825775
 id    xcenter       ycenter       aperture_sum    aperture_sum_err
         pix           pix                                         
--- ------------- ------------- ----------------- -----------------
  1 1016.86500593 1028.96463443 300868.6412045233 490.1951667229769
0012
 id     xcentroid          ycentroid            fwhm            sharpness           roundness              pa         npix    sky          peak            flux              m

 id     xcentroid          ycentroid             fwhm           sharpness           roundness              pa         npix    sky           peak             flux               mag        
--- ------------------ ------------------ ----------------- ------------------ ------------------- ------------------ ---- ---------- --------------- ------------------ ------------------
252 1015.2280280962015 1030.2119186154434 2.954102442486592 0.6564672094414649 0.03811364456571202 28.858378232843922   25 -307.30762 9573.4443359375 140376.53356933594 -12.86823628456403
 id   xcenter       ycenter       aperture_sum     aperture_sum_err 
        pix           pix                                           
--- ------------ ------------- ------------------ ------------------
  1 1015.2280281 1030.21191862 223393.75581642648 461.17972483847456
0022
 id     xcentroid          ycentroid             fwhm           sharpness           roundness               pa         npix   sky          peak            

 id     xcentroid          ycentroid            fwhm            sharpness           roundness              pa         npix    sky          peak             flux               mag        
--- ------------------ ----------------- ------------------ ------------------ ------------------- ------------------ ---- --------- --------------- ----------------- -------------------
211 1013.8335850667886 1031.956355978859 2.9960723556479323 0.6657938568106516 0.01487445125869232 10.249306311877277   25 319.15332 7537.0068359375 114634.5068359375 -12.648288417391605
 id    xcenter       ycenter       aperture_sum     aperture_sum_err
         pix           pix                                          
--- ------------- ------------- ------------------ -----------------
  1 1013.83358507 1031.95635598 217694.64393719402 457.5583937491773
0032
 id     xcentroid          ycentroid            fwhm           sharpness           roundness               pa        npix    sky          peak             flu

 id     xcentroid          ycentroid             fwhm           sharpness           roundness              pa         npix    sky         peak             flux                mag        
--- ------------------ ------------------ ----------------- ------------------ ------------------- ------------------ ---- --------- -------------- ------------------ -------------------
210 1012.8640713355803 1032.2407795591866 2.977141883829433 0.6615870852954295 0.02775421978358369 176.07506125207448   25 1159.9639 6568.083984375 101231.31396484375 -12.513287184879802
 id    xcenter       ycenter      aperture_sum    aperture_sum_err
         pix           pix                                        
--- ------------- ------------- ---------------- -----------------
  1 1012.86407134 1032.24077956 262226.788411155 475.1058759566281
0042
 id     xcentroid          ycentroid            fwhm            sharpness           roundness               pa        npix    sky          peak             flux      

...

 id     xcentroid         ycentroid             fwhm            sharpness           roundness               pa         npix    sky        peak          flux              mag        
--- ----------------- ------------------ ------------------ ------------------ -------------------- ------------------ ---- ---------- ---------- --------------- -------------------
641 966.0984640650692 1041.8795248034655 2.9579028787298873 0.6573117508288638 0.023138217355779978 1.4283598003236597   25 -4256.1836 33905.0625 451062.30078125 -14.135591327072278
 id    xcenter      ycenter       aperture_sum   aperture_sum_err
         pix          pix                                        
--- ------------- ------------ ----------------- ----------------
  1 966.098464065 1041.8795248 514527.9678688579 567.197797803281
0262
 id     xcentroid         ycentroid             fwhm           sharpness           roundness              pa        npix    sky          peak             flux               mag        


 id     xcentroid         ycentroid             fwhm            sharpness           roundness               pa        npix    sky         peak            flux               mag        
--- ----------------- ------------------ ------------------ ------------------ -------------------- ----------------- ---- --------- -------------- ---------------- -------------------
673 965.0663820126938 1041.9640385819368 2.9613871099247286 0.6580860244277175 0.004764614751397291 16.50838971952971   25 -5553.047 38784.11328125 507969.857421875 -14.264594855789817
 id    xcenter       ycenter       aperture_sum    aperture_sum_err
         pix           pix                                         
--- ------------- ------------- ----------------- -----------------
  1 965.066382013 1041.96403858 524808.0772225901 571.1713036872324
0272
 id     xcentroid         ycentroid             fwhm            sharpness           roundness               pa        npix    sky          peak             flux        

 id     xcentroid         ycentroid             fwhm            sharpness           roundness               pa        npix    sky          peak           flux              mag        
--- ----------------- ------------------ ------------------ ------------------ -------------------- ----------------- ---- ---------- -------------- -------------- -------------------
689 964.0396408226601 1042.8342329819177 2.9476398237102925 0.6550310719356206 0.017331707377406805 171.7087738538054   25 -6011.8086 39363.33203125 511522.4296875 -14.272161704484773
 id    xcenter       ycenter       aperture_sum     aperture_sum_err
         pix           pix                                          
--- ------------- ------------- ------------------ -----------------
  1 964.039640823 1042.83423298 495581.43672713754 559.3599130959901
0282
 id     xcentroid         ycentroid            fwhm            sharpness           roundness               pa        npix    sky         peak           flux           

 id     xcentroid         ycentroid             fwhm           sharpness           roundness               pa         npix    sky         peak           flux              mag        
--- ----------------- ------------------ ----------------- ------------------ -------------------- ------------------ ---- --------- -------------- -------------- -------------------
719 961.8668927972444 1041.1474582915102 2.927766958103894 0.6506148795786432 0.023618739782415257 141.83437626481478   25 -9190.727 45840.97265625 568870.7109375 -14.387533935290964
 id    xcenter       ycenter       aperture_sum    aperture_sum_err
         pix           pix                                         
--- ------------- ------------- ----------------- -----------------
  1 961.866892797 1041.14745829 419706.1105609155 527.4672583426537
0292
 id     xcentroid         ycentroid            fwhm           sharpness            roundness               pa        npix   sky         peak             flux              mag

 id    xcentroid         ycentroid             fwhm           sharpness           roundness               pa         npix    sky         peak            flux               mag        
--- ---------------- ------------------ ----------------- ------------------ -------------------- ------------------ ---- --------- ------------- ----------------- -------------------
687 960.187190730523 1040.8912547117936 2.944036548647258 0.6542303441438351 0.005433707927878864 149.01154561408075   25 -7024.824 40240.6171875 529274.3955078125 -14.309202212029767
 id    xcenter       ycenter       aperture_sum     aperture_sum_err
         pix           pix                                          
--- ------------- ------------- ------------------ -----------------
  1 960.187190731 1040.89125471 471096.73603688565 548.2286383328806
0302
 id     xcentroid         ycentroid             fwhm            sharpness           roundness               pa         npix    sky         peak           flux         